# Co-mod-gan 




## Fine tune the model on custom data

Switch to tensorflow 1.x 

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Clone the github repository

In [2]:
!git clone https://github.com/zsyzzsoft/co-mod-gan.git

Cloning into 'co-mod-gan'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 99 (delta 34), reused 86 (delta 21), pack-reused 0
Unpacking objects: 100% (99/99), done.


Zip the image files and save it in Google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd co-mod-gan/

/content/co-mod-gan


In [5]:
!unzip "/content/drive/MyDrive/Colab Notebooks/escher_images.zip" -d /content/Escher/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Escher/Escher Images/1575.png  
  inflating: /content/Escher/Escher Images/1576.png  
  inflating: /content/Escher/Escher Images/1577.png  
  inflating: /content/Escher/Escher Images/1578.png  
  inflating: /content/Escher/Escher Images/1579.png  
  inflating: /content/Escher/Escher Images/158.png  
  inflating: /content/Escher/Escher Images/1580.png  
  inflating: /content/Escher/Escher Images/1581.png  
  inflating: /content/Escher/Escher Images/1582.png  
  inflating: /content/Escher/Escher Images/1583.png  
  inflating: /content/Escher/Escher Images/1584.png  
  inflating: /content/Escher/Escher Images/1585.png  
  inflating: /content/Escher/Escher Images/1586.png  
  inflating: /content/Escher/Escher Images/1587.png  
  inflating: /content/Escher/Escher Images/1588.png  
  inflating: /content/Escher/Escher Images/1589.png  
  inflating: /content/Escher/Escher Images/159.png  
  inflating: /content/Escher/Esch

Make nevessary folders and move the validation set. Here I am preserving 100 images for validation

In [6]:
!mkdir ./data/
!mkdir ./data/validation/
!mkdir ./data/training/
!mv /content/Escher/Escher\ Images/9??.png ./data/validation/
!mv /content/Escher/Escher\ Images/* ./data/training/
!rm -rf /content/Escher/

rm: cannot remove '/content/escher_images.zip': No such file or directory


Resize all the images to (512,512,3)

In [7]:
import cv2
import numpy as np
import glob
image=glob.glob('/content/co-mod-gan/data/validation/*')
image.extend(glob.glob('/content/co-mod-gan/data/training/*'))

for i in image:
  im1=cv2.imread(i)
  if im1.shape!=(512,512,3):
    resize=cv2.resize(im1,(512,512))
    cv2.imwrite(i,resize)


Move the model from the drive to a folder in pwd to fine tune it

In [9]:
!cp /content/drive/MyDrive/co-mod-gan-places2-050000.pkl ./model

In [13]:
!mkdir escher
!mv ./data/data-r09.tfrecords ./escher/

Create the tfrecords dataset of the images

In [29]:
!python dataset_tools/create_from_images.py --tfrecord-dir ./data --val-image-dir ./data/validation --train-image-dir ./data/training

Processes created.



Processing validation images...
./data/validation
100% 100/100 [00:00<00:00, 77903.12it/s]
100% 100/100 [00:06<00:00, 14.94it/s]
Processing training images...
./data/training
100% 5541/5541 [00:00<00:00, 110632.30it/s]
100% 5541/5541 [06:07<00:00, 15.08it/s]


Train the model

In [48]:
!python run_training.py --data-dir ./ --dataset data/ --metrics=ids36k5 --total-kimg 0 --num-gpus=1 --resume /content/co-mod-gan/model/co-mod-gan-places2-050000.pkl


Local submit - run_dir: results/00028-co-mod-gan--1gpu
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x559f358ea000 @  0x7f79e5258001 0x7f79e1d3c54f 0x7f79e1d8cb58 0x7f79e1d90b17 0x7f79e1e2f203 0x559f2ed5b0e4 0x559f2ed5ade0 0x559f2edcf6f5 0x559f2edc9e0d 0x559f2ed5d02c 0x559f2ed9dd39 0x559f2ed9ac84 0x559f2ed5d231 0x559f2edcc1e6 0x559f2edc9b0e 0x559f2ec9be2b 0x559f2edcc1e6 0x559f2edc9e0d 0x559f2ec9be2b 0x559f2edcc1e6 0x559f2ed5c69a 0x559f2edcaa45 0x559f2ed5c69a 0x559f2edcac9e 0x559f2edc9b0e 0x559f2ec9be2b 0x559f2edcc1e6 0x559f2edc9b0e 0x559f2ec9be2b 0x559f2edcc1e6 0x559f2ed5c69a
tcmalloc: large alloc 4294967296 bytes == 0x55a0358ec000 @  0x7f79e52561e7 0x7f79e1d3c46e 0x7f79e1d8cc7b 0x7f79e1d8d35f 0x7f79e1e2f103 0x559f2ed5b0e4 0x559f2ed5ade0 0x559f2edcf6f5 0x559f2edc9b0e 0x559f2ed5c77a 0x559f2edcb86a 0x559f2edc9b0e 0x559f2ed5c77a 0x559f2edcb86a 0x559f2edc9b0e 0x559f2

## Test the model on model

Add your input image and mask to the `imgs` folder and run the below cell. The output will be saved as `output.jpg`

In [28]:
!python run_generator.py -c /content/drive/MyDrive/co-mod-gan-places2-050000.pkl -i imgs/image.jpg -m imgs/mask.jpg -o output.jpg

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
